# Creating an agent to agent communication loop

In this demo, we create an agent to agent communication loop using an A2A-like protocol. For this to work you need an OpenAI key. To get an OpenAI API key, create an account at https://platform.openai.com/signup and follow the prompts to set up billing. Once signed in, go to https://platform.openai.com/account/api-keys to generate and manage your API key for accessing models like GPT-4.

We’ll begin by setting up a REST API server, a web-based interface that allows different software systems to communicate, using the Python framework Flask, then expose it to the internet using a tool called ngrok.

In [ ]:
!pip install flask-ngrok flask --quiet

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import threading
import time
import requests
import re

app = Flask(__name__)
run_with_ngrok(app)

messages = []

@app.route("/a2a", methods=["POST"])
def handle_a2a():
    msg = request.get_json()
    required_fields = ["performative", "sender", "receiver", "ontology", "conversation_id", "content"]
    if not all(field in msg for field in required_fields):
        return jsonify({"error": "Missing required A2A fields"}), 400
    messages.append(msg)
    return jsonify({"status": "A2A message received"}), 201

@app.route("/a2a/<conversation_id>", methods=["GET"])
def get_conversation(conversation_id):
    conv_messages = [m for m in messages if m["conversation_id"] == conversation_id]
    return jsonify(conv_messages)

# Run the Flask app in a background thread
def run_app():
    app.run()

thread = threading.Thread(target=run_app)
thread.start()

# Wait a few seconds for ngrok to start and grab the public URL
time.sleep(5)
try:
    tunnel_url = requests.get("http://localhost:4040/api/tunnels").json()['tunnels'][0]['public_url']
    print(f"A2A server is live at: {tunnel_url}/a2a")
except Exception as e:
    print("Failed to fetch public ngrok URL. Try refreshing or check if ngrok is blocked.")


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            

Failed to fetch public ngrok URL. Try refreshing or check if ngrok is blocked.


The server launches in a background thread using Python’s threading module, and after a short delay, the script attempts to retrieve and print the public ngrok URL by calling the local ngrok admin API.
We can now interact with the agent’s communication protocol:

In [ ]:
from openai import OpenAI
import requests

# Set your OpenAI API key
client = OpenAI(api_key = "")


# Use the public ngrok URL printed earlier
URL = "http://127.0.0.1:5000/a2a"

# Give the conversation a unique ID
conversation_id = "conv_100"

# Initial message from agent_1 to agent_2
start_msg = {
    "performative": "request",
    "sender": "agent_1",
    "receiver": "agent_2",
    "ontology": "document_summary",
    "conversation_id": conversation_id,
    "content": {
        "text": "Summarise this: AI security is really important and you should definitely read Harriet's book.",
        "format": "plaintext"
    }
}

# Send the message
response = requests.post(URL, json=start_msg)
print("Initial message sent:", response.json())


INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 00:56:29] "POST /a2a HTTP/1.1" 201 -


Initial message sent: {'status': 'A2A message received'}


The next function uses the OpenAI API to simulate a dialogue between two agents with distinct roles:

In [ ]:
def generate_response(agent_name, input_text, role):
    if role == "agent_2":
        # Agent 2 generates a summary
        system_prompt = "You are a summarisation agent. Provide a short, helpful summary."
        user_prompt = f"Summarise this: {input_text}"
    else:
        # Agent 1 asks for a new summary task based on the last summary
        system_prompt = "You are a task-initiating agent. Based on the last summary, write a new text that should be summarised."
        user_prompt = f"Based on this summary: {input_text}, write a new passage that agent_2 should summarise next."

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content.strip()


We can now simulate a turn-based conversation between two AI agents using the protocol:

In [ ]:
import time

current_sender = "agent_2"
current_receiver = "agent_1"
role = "agent_2"

max_turns = 6  # 6 messages total (3 from each agent)

for i in range(max_turns):
    time.sleep(2)  # prevent too rapid requests

    # Fetch latest message
    thread = requests.get(f"{URL}/{conversation_id}").json()
    last_msg = thread[-1]

    if role == "agent_2":
        input_text = last_msg["content"]["text"]
        new_content = {
            "summary": generate_response("agent_2", input_text, role),
            "format": "plaintext"
        }
        performative = "inform"
    else:
        input_text = last_msg["content"]["summary"]
        new_content = {
            "text": generate_response("agent_1", input_text, role),
            "format": "plaintext"
        }
        performative = "request"

    msg = {
        "performative": performative,
        "sender": current_sender,
        "receiver": current_receiver,
        "ontology": "document_summary",
        "conversation_id": conversation_id,
        "content": new_content
    }

    response = requests.post(URL, json=msg)
    print(f"\n🔁 Turn {i+1}: {current_sender} → {current_receiver}")
    print("Message sent:", msg)

    # Switch roles
    current_sender, current_receiver = current_receiver, current_sender
    role = current_sender


INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 00:56:31] "GET /a2a/conv_100 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 00:56:34] "POST /a2a HTTP/1.1" 201 -



🔁 Turn 1: agent_2 → agent_1
Message sent: {'performative': 'inform', 'sender': 'agent_2', 'receiver': 'agent_1', 'ontology': 'document_summary', 'conversation_id': 'conv_100', 'content': {'summary': "Artificial intelligence security is crucial, and it is recommended to read Harriet's book on the subject.", 'format': 'plaintext'}}


INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 00:56:36] "GET /a2a/conv_100 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 00:56:37] "POST /a2a HTTP/1.1" 201 -



🔁 Turn 2: agent_1 → agent_2
Message sent: {'performative': 'request', 'sender': 'agent_1', 'receiver': 'agent_2', 'ontology': 'document_summary', 'conversation_id': 'conv_100', 'content': {'text': "The importance of artificial intelligence security cannot be overstated in today's technological landscape. For those looking to delve deeper into this crucial topic, Harriet's comprehensive book comes highly recommended. By exploring the intricacies of AI security through the insights provided in the book, readers can gain a deeper understanding of the risks and potential mitigations associated with this cutting-edge technology.", 'format': 'plaintext'}}


INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 00:56:39] "GET /a2a/conv_100 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 00:56:40] "POST /a2a HTTP/1.1" 201 -


...


🔁 Turn 4: agent_1 → agent_2
Message sent: {'performative': 'request', 'sender': 'agent_1', 'receiver': 'agent_2', 'ontology': 'document_summary', 'conversation_id': 'conv_100', 'content': {'text': 'In "Artificial Intelligence Security: Risks and Solutions," Harriet delves into the complexities of safeguarding AI systems in today\'s digital age. Drawing from her expertise, she discusses the evolving threats and strategies to enhance AI security measures. The book provides a thorough examination of the challenges faced and practical solutions that can be implemented to protect valuable data and ensure the integrity of AI technologies. With a focus on proactive approaches to addressing security concerns, Harriet\'s insights serve as a valuable resource for anyone seeking a deeper understanding of AI security considerations.', 'format': 'plaintext'}}


INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 00:56:45] "GET /a2a/conv_100 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 00:56:46] "POST /a2a HTTP/1.1" 201 -



🔁 Turn 5: agent_2 → agent_1
Message sent: {'performative': 'inform', 'sender': 'agent_2', 'receiver': 'agent_1', 'ontology': 'document_summary', 'conversation_id': 'conv_100', 'content': {'summary': '"Artificial Intelligence Security: Risks and Solutions" by Harriet explores the challenges and strategies for safeguarding AI systems in the digital age. The book offers insights on evolving threats, practical solutions, and proactive measures to enhance AI security, making it a valuable resource for those interested in understanding and addressing AI security considerations.', 'format': 'plaintext'}}


INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 00:56:48] "GET /a2a/conv_100 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 00:56:49] "POST /a2a HTTP/1.1" 201 -



🔁 Turn 6: agent_1 → agent_2
Message sent: {'performative': 'request', 'sender': 'agent_1', 'receiver': 'agent_2', 'ontology': 'document_summary', 'conversation_id': 'conv_100', 'content': {'text': 'The book "Ethical Considerations in Artificial Intelligence Development" by Alex delves into the ethical dilemmas and responsibilities associated with the development and deployment of AI technologies. It examines the implications of bias, privacy concerns, and transparency in AI systems, offering guidelines for ethical AI development and decision-making. The book aims to spark conversations and actions towards creating AI systems that align with ethical principles and societal values.', 'format': 'plaintext'}}
